# Fase PreDB

El objetivo es preparar las tablas para nuestra Base de Datos

In [54]:
# Librerías para el tratamiento y manipulación de datos
# -----------------------------------------------------------------------
import pandas as pd             # Para manipular estructuras de datos como DataFrames
import numpy as np              # Para cálculos numéricos y arrays multidimensionales
import datetime                 # Para manejar fechas y horas

import os
import sys
sys.path.append("../")


In [55]:
df_gas = pd.read_csv("../datos/01_PreciosGas/03_TablaPreciosGasFinal.csv",index_col="Unnamed: 0")
df_luz = pd.read_csv("../datos/02_PreciosLuz/02_PreciosLuzFinal.csv",index_col="Unnamed: 0")
df_combustible = pd.read_csv("../datos/03_PreciosCombustible/01_TablaPreciosCombustible.csv",index_col="Unnamed: 0")
df_agua = pd.read_csv("../datos/04_PreciosAgua/01_PreciosAguaFinal.csv",index_col="Unnamed: 0")

In [56]:
df_gas.sample()

,index,precio_gas,fecha
40,41,77.03,2022-05-01


In [57]:
df_luz.sample()

,index,precio,fecha
9,10,52.17,2019-10-01


In [58]:
df_combustible = df_combustible[["combustible","Precio","Fecha"]]
df_combustible = df_combustible.rename(columns={"Precio":"precio",
                                                "Fecha":"fecha"})
df_combustible.sample()

,combustible,precio,fecha
157,Gasolina 98,1.775861,2022-02-01


In [59]:
df_agua = df_agua.drop(columns=["ciudad"])
years = df_agua['año'].tolist()
precios = df_agua['consumo (€/m3)'].tolist()
fechas = []
precios_mes = []
for year, price in zip(years, precios):
    fechas.extend(pd.date_range(start=f'{year}-01-01', periods=12, freq='MS'))
    precios_mes.extend([price] * 12)
    
df_agua = pd.DataFrame({
    'fecha': fechas,
    'consumo (€/m3)': precios_mes
})


In [61]:
df_agua.to_csv("../datos/05_TablasPreDB/ex.csv")